In [1]:
import numpy as np
from numpy import pi
import matplotlib.pyplot as plt
%matplotlib inline
from astropy import units as u, constants as ac
from scipy import constants as sc
import sympy as sp
sp.init_printing(use_latex='mathjax')

In [2]:
from IPython.core.display import display, HTML, Latex
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def show(*args):
    for arg in args:
        if isinstance(arg, str):
            print(arg, end=" ")
        else:
            print("%e" % arg, end=" ")
    print("")

In [4]:
C, f_c, f_a, Lambda_QCD, kappa, N, rho_c, g_0, g_osc, T_0, gamma, theta_i, eV = \
 sp.symbols("C, f_c, f_a, Lambda_QCD, kappa, N, rho_c, g_0, g_osc, T_0, gamma, theta_i, eV")
GeV = 10**9 * eV
T_osc = kappa * (10**16 * GeV / (f_a / N))**(sp.Rational(1, 6))
m_a = 6*10**(-10) * eV * 10**16 * GeV / (f_a / N)
zeta = C * (Lambda_QCD / T_osc)**4 # ok
m_a_osc = m_a * zeta # ok
# s up to a constant factor
s0 = g_0 * T_0**3 # ok
s_osc = g_osc * T_osc**3 # ok
n_a = f_c / 2 * m_a_osc * (f_a / N)**2 * theta_i**2 
rho_a = n_a * m_a * s0 / s_osc * gamma
Omega_a = rho_a / rho_c # ok
Omega_a 

       5/6        4   3   4            2
18.0⋅10   ⋅C⋅Λ_QCD ⋅T₀ ⋅eV ⋅f_c⋅g₀⋅γ⋅θᵢ 
────────────────────────────────────────
                            7/6         
                7     ⎛N⋅eV⎞            
         g_osc⋅κ ⋅ρ_c⋅⎜────⎟            
                      ⎝ fₐ ⎠            

In [40]:
M_p = sc.physical_constants["Planck mass"][0] * sc.c**2 / sc.elementary_charge
C = 0.018
g_star = 61.75
lambda_QCD = sp.RR(200 * 1e6)
kappa = sp.RR(((10 * M_p**2 / (pi**2 * g_star))**(1/4) * C**(1/2) * lambda_QCD**2 * (6e-6 / 1e4)**(1/2))**(1/3))
g_star_s0 = 3.91
g_star_s_osc = g_star 
T0 = 2.73 * sc.Boltzmann / sc.elementary_charge
f_c = 1.44
H0 = sp.RR(((67.74 * u.km / u.second / u.Mpc * ac.hbar).to("eV") / u.eV).to_value())
M_pl = sp.RR(2.435e18) * 1e9 # [eV]
rho_c = 1 / (3 * H0**2 * M_pl**2)
h = 0.67 
iota = 18 * 10 ** (5 / 6) * C * lambda_QCD**4 * T0**3 * f_c * g_star_s0 / (g_star_s_osc * kappa**7 * rho_c)
show("iota =", iota / h**2, ", kappa =", kappa)
iota * (1e16 * 1e9)**(7/6) / h**2

iota = 7.409887e-47 , kappa = 1.732314e+08 


1.08762266253743e-17

In [37]:
(2.73 * u.K * ac.k_B).to("eV"), T0

(<Quantity 0.00023525 eV>, 0.00023525311820602992)